In [1]:

import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

import numpy as np
import torch
#import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# from sklearn.model_selection import train_test_split
import tqdm
import matplotlib.pyplot as plt
import logging

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.chdir('../')

from pose_gru import PoseGRU_inputFC2
from benji_prox_dataloader import *

numpy: 1.21.5


In [2]:
root_dir = "D:/prox_data/joint_locations/"
# root_dir = "/cluster/scratch/bdayan/prox_data/joint_locations/"

batch_size = 15

pd = proxDatasetJoints(root_dir)
dataloader = DataLoader(pd, batch_size=batch_size,
                        shuffle=True, num_workers=0, collate_fn=my_collate)

In [3]:
os.listdir('model_saves')

['model_epoch0_bn391.pt',
 'model_epoch1_bn391.pt',
 'model_epoch2_bn391.pt',
 'model_epoch3_bn391.pt',
 'model_epoch4_bn391.pt',
 'model_epoch5_bn391.pt',
 'model_epoch6_bn391.pt',
 'model_epoch7_bn391.pt',
 'model_epoch8_bn391.pt']

In [4]:
temp = torch.load('model_saves/model_epoch0_bn391.pt')

In [5]:
temp

{'epoch': 0,
 'batch_num': 391,
 'model_state_dict': OrderedDict([('input_fc.weight',
               tensor([[-0.0778, -0.0539, -0.1031,  ...,  0.0457, -0.1046,  0.0689],
                       [ 0.0752,  0.1063, -0.0152,  ...,  0.0983,  0.0975,  0.0616],
                       [ 0.0906, -0.1042,  0.0630,  ...,  0.0733,  0.0256, -0.0575],
                       ...,
                       [-0.0036, -0.0701, -0.1068,  ...,  0.0971,  0.0305,  0.0513],
                       [-0.0931,  0.0694,  0.0517,  ...,  0.0021, -0.0607,  0.0622],
                       [-0.0267,  0.0743,  0.0418,  ..., -0.0525, -0.0583,  0.0038]])),
              ('input_fc.bias',
               tensor([-7.7283e-02,  5.6047e-02,  8.1545e-02, -4.4304e-02, -8.1215e-02,
                        7.6695e-02, -6.9363e-02, -1.0661e-01,  2.1687e-02,  8.9244e-02,
                        5.2472e-03,  3.4373e-02, -3.9705e-02, -3.7916e-02, -1.1000e-01,
                       -1.0352e-01,  5.1028e-02,  3.5083e-02, -8.0592e-02,  1

In [6]:
gru = PoseGRU_inputFC2(input_size=(25,3))
gru.load_state_dict(temp)

RuntimeError: Error(s) in loading state_dict for PoseGRU_inputFC2:
	Missing key(s) in state_dict: "input_fc.weight", "input_fc.bias", "GRUcell_list.0.weight_ih", "GRUcell_list.0.weight_hh", "GRUcell_list.0.bias_ih", "GRUcell_list.0.bias_hh", "GRUcell_list.1.weight_ih", "GRUcell_list.1.weight_hh", "GRUcell_list.1.bias_ih", "GRUcell_list.1.bias_hh", "fc1.weight", "fc1.bias". 
	Unexpected key(s) in state_dict: "epoch", "batch_num", "model_state_dict", "optimizer_state_dict", "loss". 

In [4]:
idx, in_joints, pred_joints = pd.__getitem__(100)

In [5]:
in_joints.shape

torch.Size([10, 25, 3])

In [9]:
pred_joints.shape

torch.Size([5, 25, 3])

In [7]:
torch.mean(in_joints[-1])

tensor(-0.0934, grad_fn=<MeanBackward0>)

In [8]:
[torch.mean(x) for x in in_joints]

[tensor(-0.0895, grad_fn=<MeanBackward0>),
 tensor(-0.0895, grad_fn=<MeanBackward0>),
 tensor(-0.0878, grad_fn=<MeanBackward0>),
 tensor(-0.0915, grad_fn=<MeanBackward0>),
 tensor(-0.0925, grad_fn=<MeanBackward0>),
 tensor(-0.0924, grad_fn=<MeanBackward0>),
 tensor(-0.0904, grad_fn=<MeanBackward0>),
 tensor(-0.0901, grad_fn=<MeanBackward0>),
 tensor(-0.0888, grad_fn=<MeanBackward0>),
 tensor(-0.0934, grad_fn=<MeanBackward0>)]